In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import umap
import hdbscan

C:\Users\Lukasz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Wczytanie danych

In [2]:
hc_training = pd.read_csv('../../../classifier_data/HC_training.csv', sep='\t').T
hc_test = pd.read_csv('../../../classifier_data/HC_test.csv', sep='\t').T
oc_training = pd.read_csv('../../../classifier_data/OC_training.csv', sep='\t').T
oc_test = pd.read_csv('../../../classifier_data/OC_test.csv', sep='\t').T

synthetic_hc_data_path = '../../../best_models/HC/WGAN/2000/synthetic_data/'
model_id = os.listdir(synthetic_hc_data_path)[0]
hc_synthetic = pd.read_csv(f'{synthetic_hc_data_path}/{model_id}/generated_data.tsv', sep='\t')

synthetic_oc_data_path = '../../../best_models/OC/WGAN/2000/synthetic_data/'
model_id = os.listdir(synthetic_oc_data_path)[0]
oc_synthetic = pd.read_csv(f'{synthetic_oc_data_path}/{model_id}/generated_data.tsv', sep='\t')

# combine training and test data with labels
hc_training['label'] = 'HC'
hc_test['label'] = 'HC'
oc_training['label'] = 'OC'
oc_test['label'] = 'OC'

hc_synthetic['label'] = 'HC'
oc_synthetic['label'] = 'OC'

training_data = pd.concat([hc_training, oc_training])
test_data = pd.concat([hc_test, oc_test])

In [3]:
# keep only the columns that are in both datasets
# Znalezienie wspólnych kolumn
common_columns = training_data.columns.intersection(test_data.columns)
common_columns = common_columns.intersection(hc_synthetic.columns)
common_columns = common_columns.intersection(oc_synthetic.columns)

# Uporządkowanie kolumn w real_data
training_data = training_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
test_data = test_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
hc_synthetic = hc_synthetic[common_columns]
oc_synthetic = oc_synthetic[common_columns]

common_columns

Index(['ENSG00000000419', 'ENSG00000000938', 'ENSG00000002549',
       'ENSG00000002822', 'ENSG00000003756', 'ENSG00000004487',
       'ENSG00000004534', 'ENSG00000005007', 'ENSG00000005059',
       'ENSG00000005302',
       ...
       'ENSG00000244734', 'ENSG00000249072', 'ENSG00000253729',
       'ENSG00000255823', 'ENSG00000257207', 'ENSG00000258890',
       'ENSG00000266714', 'ENSG00000269028', 'ENSG00000271043', 'label'],
      dtype='object', length=1548)

In [4]:
# add synthetic data to training data
training_data_synthetic = pd.concat([training_data, hc_synthetic, oc_synthetic])
training_data_synthetic

,ENSG00000000419,ENSG00000000938,ENSG00000002549,ENSG00000002822,ENSG00000003756,ENSG00000004487,ENSG00000004534,ENSG00000005007,ENSG00000005059,ENSG00000005302,...,ENSG00000244734,ENSG00000249072,ENSG00000253729,ENSG00000255823,ENSG00000257207,ENSG00000258890,ENSG00000266714,ENSG00000269028,ENSG00000271043,label
VUMC-HC-0033-TR2591,3.635790,4.432779,4.954165,3.720018,4.670109,4.211676,4.722403,6.139299,4.399111,4.169415,...,6.995484,6.002046,4.866702,6.095028,9.847484,4.211676,4.465409,7.757440,6.525607,HC
Vumc-HD-70-TR1062,4.546459,5.786592,6.072522,4.884990,5.279119,5.309622,5.479209,5.309622,5.828302,5.215895,...,10.021953,5.215895,6.019849,5.215895,8.262227,4.601320,4.139052,6.645036,5.397116,HC
VUMC-HC0053-DOT-HD-48h-TR3087,3.861049,5.526849,4.724280,4.253528,4.467036,4.292393,4.253528,5.631686,4.913257,4.498734,...,7.050183,3.861049,4.724280,3.861049,8.831693,3.861049,4.030369,4.253528,3.720811,HC
Vumc-HD-149-TR932,4.727528,4.923042,4.945413,3.937986,3.647372,4.727528,3.808559,5.031082,4.673182,4.829242,...,8.661739,6.247257,5.275333,6.360425,3.647372,3.083092,3.808559,8.517492,6.826289,HC
Vumc-HD-36-TR1165,4.402386,6.484772,4.751692,4.172174,4.402386,5.811822,7.053716,4.402386,5.998264,5.241598,...,9.877834,5.515912,6.055240,5.241598,3.862107,5.339481,6.355612,7.025844,5.339481,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,4.314255,4.430532,4.434036,3.857375,3.821334,4.896678,3.501221,4.723317,3.823658,3.734970,...,8.739038,3.966150,4.700540,4.656014,7.042653,4.581230,3.768886,5.049108,4.496623,OC
77,4.334697,5.069236,4.162939,4.718737,4.169561,4.497835,4.017847,5.924613,3.479529,3.687467,...,11.059032,5.183362,4.517694,4.440514,7.242820,4.752573,3.837408,4.932785,6.107358,OC
78,4.401705,4.463482,4.796886,4.276219,4.368454,4.571169,4.442346,6.560488,4.055482,4.694121,...,9.049995,5.490667,5.048331,5.056717,7.551162,4.526353,4.285269,6.225359,6.299684,OC
79,3.907023,4.023447,4.392516,4.057934,3.591298,4.288187,3.840995,5.463649,3.583701,3.825759,...,11.715482,4.358302,4.061844,4.306998,7.367720,4.352803,3.789771,5.808754,6.151889,OC


### Wykorzystanie klasyfikatorów

In [5]:
# klasyfikacja z uyciem svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


def svm_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Klasyfikacja przy użyciu SVM
    svm = SVC(kernel='linear', random_state=42)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


svm_classification(training_data, test_data)
svm_classification(training_data_synthetic, test_data)

Dokładność: 0.9405405405405406
              precision    recall  f1-score   support

          HC       0.93      0.99      0.96       131
          OC       0.98      0.81      0.89        54

    accuracy                           0.94       185
   macro avg       0.95      0.90      0.92       185
weighted avg       0.94      0.94      0.94       185

[[130   1]
 [ 10  44]]
Dokładność: 0.9351351351351351
              precision    recall  f1-score   support

          HC       0.93      0.98      0.96       131
          OC       0.96      0.81      0.88        54

    accuracy                           0.94       185
   macro avg       0.94      0.90      0.92       185
weighted avg       0.94      0.94      0.93       185

[[129   2]
 [ 10  44]]


In [6]:
from sklearn.tree import DecisionTreeClassifier
# klasyfikacja z uyciem random forest
def decision_tree_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    dt = DecisionTreeClassifier(max_depth=5, random_state=42)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

decision_tree_classification(training_data, test_data)
decision_tree_classification(training_data_synthetic, test_data)

Dokładność: 0.7891891891891892
              precision    recall  f1-score   support

          HC       0.85      0.85      0.85       131
          OC       0.64      0.63      0.64        54

    accuracy                           0.79       185
   macro avg       0.74      0.74      0.74       185
weighted avg       0.79      0.79      0.79       185

[[112  19]
 [ 20  34]]
Dokładność: 0.7567567567567568
              precision    recall  f1-score   support

          HC       0.82      0.85      0.83       131
          OC       0.59      0.54      0.56        54

    accuracy                           0.76       185
   macro avg       0.70      0.69      0.70       185
weighted avg       0.75      0.76      0.75       185

[[111  20]
 [ 25  29]]


In [7]:
from sklearn.ensemble import RandomForestClassifier
# klasyfikacja z uyciem random forest
def random_forest_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

random_forest_classification(training_data, test_data)
random_forest_classification(training_data_synthetic, test_data)

Dokładność: 0.8378378378378378
              precision    recall  f1-score   support

          HC       0.82      0.99      0.90       131
          OC       0.96      0.46      0.62        54

    accuracy                           0.84       185
   macro avg       0.89      0.73      0.76       185
weighted avg       0.86      0.84      0.82       185

[[130   1]
 [ 29  25]]
Dokładność: 0.8702702702702703
              precision    recall  f1-score   support

          HC       0.85      1.00      0.92       131
          OC       1.00      0.56      0.71        54

    accuracy                           0.87       185
   macro avg       0.92      0.78      0.82       185
weighted avg       0.89      0.87      0.86       185

[[131   0]
 [ 24  30]]


In [8]:
from xgboost import XGBClassifier

# klasyfikacja z uyciem xgboost
def xgboost_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    y_train = y_train.replace('HC', 0)
    y_train = y_train.replace('OC', 1)
    y_test = y_test.replace('HC', 0)
    y_test = y_test.replace('OC', 1)
    xgb = XGBClassifier(random_state=42)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

xgboost_classification(training_data, test_data)
xgboost_classification(training_data_synthetic, test_data)

Dokładność: 0.9081081081081082
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       131
           1       0.93      0.74      0.82        54

    accuracy                           0.91       185
   macro avg       0.92      0.86      0.88       185
weighted avg       0.91      0.91      0.90       185

[[128   3]
 [ 14  40]]
Dokładność: 0.9243243243243243
              precision    recall  f1-score   support

           0       0.91      0.98      0.95       131
           1       0.95      0.78      0.86        54

    accuracy                           0.92       185
   macro avg       0.93      0.88      0.90       185
weighted avg       0.93      0.92      0.92       185

[[129   2]
 [ 12  42]]


In [9]:
from sklearn.neural_network import MLPClassifier

# klasyfikacja z uzyciem sieci neuronowych

def nn_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    nn = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42)
    nn.fit(X_train, y_train)
    y_pred = nn.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

nn_classification(training_data, test_data)
nn_classification(training_data_synthetic, test_data)

Dokładność: 0.9297297297297298
              precision    recall  f1-score   support

          HC       0.93      0.98      0.95       131
          OC       0.94      0.81      0.87        54

    accuracy                           0.93       185
   macro avg       0.93      0.90      0.91       185
weighted avg       0.93      0.93      0.93       185

[[128   3]
 [ 10  44]]
Dokładność: 0.9297297297297298
              precision    recall  f1-score   support

          HC       0.93      0.97      0.95       131
          OC       0.92      0.83      0.87        54

    accuracy                           0.93       185
   macro avg       0.93      0.90      0.91       185
weighted avg       0.93      0.93      0.93       185

[[127   4]
 [  9  45]]
